In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

In [2]:
df=pd.read_parquet("ori_df.parquet")
df.columns

Index(['Tail_Number', 'prev_arr_dt', 'prev_arr_delay', 'prev_origin',
       'prev_dest', 'prev_crs_dep', 'prev_crs_arr', 'next_dep_dt', 'DepDelay',
       'next_origin', 'Dest', 'next_crs_dep', 'next_crs_arr', 'turn_time_min',
       'O-D', 'historical_turn_time_min_3mean', 'scheduled_turn_time',
       'realized slack', 'scheturn_bucket', 'prev_bin', 'op_carrier',
       'op_flight', 'Origin', 'carrier_slope', 'origin_slope', 'crsdep_hour',
       'dep_hour', 'prev_arr_hour', 'datetime', 'temperature_2m',
       'wind_speed_10m', 'wind_gusts_10m', 'precipitation', 'is_day',
       'cloud_cover', 'airport', 'datetime_pa', 'temperature_2m_pa',
       'wind_speed_10m_pa', 'wind_gusts_10m_pa', 'precipitation_pa',
       'is_day_pa', 'cloud_cover_pa', 'airport_pa', 'Distance',
       'DistanceGroup', 'CRSElapsedTime', 'DayOfWeek', 'dep_dt', 'FlightDate',
       'crsdep_tbin', 'prev_arr_tbin'],
      dtype='object')

Features Engineering:

In [ ]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

import optuna


y_label = "DepDelay"

numeric_cols = [
    'prev_arr_delay',
    'historical_turn_time_min_3mean',
    'scheduled_turn_time',
    'realized slack',
    'carrier_slope',
    'origin_slope',
    'temperature_2m',
    'wind_speed_10m',
    'wind_gusts_10m',
    'precipitation',
    'cloud_cover',
    'temperature_2m_pa',
    'wind_speed_10m_pa',
    'wind_gusts_10m_pa',
    'precipitation_pa',
    'cloud_cover_pa',
    'Distance',
    'CRSElapsedTime',
]

cat_cols = [
    'prev_origin',
    'Dest',
    'scheturn_bucket',
    'prev_bin',
    'op_carrier',
    'Origin',
    'prev_arr_tbin',
    'crsdep_tbin',
    'is_day',
    'is_day_pa',
    'DistanceGroup',
    'DayOfWeek'
]


def add_advanced_features(df):
    df = df.copy()
    df["FlightDate"] = pd.to_datetime(df["FlightDate"])
    df = df.sort_values("FlightDate")


    df["slack_time"] = df["scheduled_turn_time"] - df["historical_turn_time_min_3mean"]

    df["prev_arr_delay_clip"] = df["prev_arr_delay"].clip(lower=-30, upper=180)

    df["scheduled_turn_capped"] = df["scheduled_turn_time"].clip(lower=5, upper=180)

    df["prop_prev_delay_over_planned_turn"] = (
        df["prev_arr_delay_clip"] / df["scheduled_turn_capped"]
    )

    df["prop_prev_delay_over_turn"] = df["prev_arr_delay_clip"] / (df["scheduled_turn_time"] + 1e-3)

    df["short_turn"] = (df["scheduled_turn_time"] < 45).astype(int)
    df["short_turn_prev_delay"] = df["short_turn"] * df["prev_arr_delay_clip"]


    df["wind_speed_diff"] = df["wind_speed_10m"] - df["wind_speed_10m_pa"]
    df["wind_gusts_diff"] = df["wind_gusts_10m"] - df["wind_gusts_10m_pa"]
    df["temp_diff"]       = df["temperature_2m"] - df["temperature_2m_pa"]
    df["precip_diff"]     = df["precipitation"] - df["precipitation_pa"]
    df["cloud_diff"]      = df["cloud_cover"] - df["cloud_cover_pa"]

    df["has_rain"]    = (df["precipitation"] > 0.1).astype(int)
    df["has_rain_pa"] = (df["precipitation_pa"] > 0.1).astype(int)


    df = df.sort_values(["Origin", "FlightDate"])
    df["origin_delay_mean_7d"] = (
        df.groupby("Origin")["DepDelay"]
          .transform(lambda x: x.shift().rolling(window=7, min_periods=3).mean())
    )

    df = df.sort_values(["op_carrier", "FlightDate"])
    df["carrier_delay_mean_7d"] = (
        df.groupby("op_carrier")["DepDelay"]
          .transform(lambda x: x.shift().rolling(window=7, min_periods=3).mean())
    )

    df["Route"] = df["Origin"].astype(str) + "_" + df["Dest"].astype(str)
    df = df.sort_values(["Route", "FlightDate"])
    df["route_delay_mean_7d"] = (
        df.groupby("Route")["DepDelay"]
          .transform(lambda x: x.shift().rolling(window=7, min_periods=5).mean())
    )

    global_mean_delay = df["DepDelay"].mean()
    for col in ["origin_delay_mean_7d", "carrier_delay_mean_7d", "route_delay_mean_7d"]:
        df[col] = df[col].fillna(global_mean_delay)


    df["avg_speed"] = df["Distance"] / (df["CRSElapsedTime"] + 1e-3)
    df["prev_big_delay"] = (df["prev_arr_delay"] >= 30).astype(int)

    return df



advanced_numeric = [
    "slack_time",
    "prev_arr_delay_clip",
    "prop_prev_delay_over_turn",
    "short_turn_prev_delay",
    "wind_speed_diff",
    "wind_gusts_diff",
    "temp_diff",
    "precip_diff",
    "cloud_diff",
    "origin_delay_mean_7d",
    "carrier_delay_mean_7d",
    "route_delay_mean_7d",
    "avg_speed",
]

advanced_cat = [
    "short_turn",
    "has_rain",
    "has_rain_pa",
    "prev_big_delay",
]

numeric_cols_extended = numeric_cols + advanced_numeric
cat_cols_extended     = cat_cols + advanced_cat
features_extended     = numeric_cols_extended + cat_cols_extended

In [11]:
pd.DataFrame(features_extended).to_csv("featurenames.csv")

In [ ]:

df = add_advanced_features(df)

df_model = df.copy()
df_model = df_model[df_model[y_label].notna()].copy()
df_model["FlightDate"] = pd.to_datetime(df_model["FlightDate"])
df_model = df_model.sort_values("FlightDate")

train_end = "2025-01-21"
valid_end = "2025-01-28"

train = df_model[df_model["FlightDate"] <= train_end].copy()
valid = df_model[(df_model["FlightDate"] > train_end) & (df_model["FlightDate"] <= valid_end)].copy()
test  = df_model[df_model["FlightDate"] > valid_end].copy()

X_train_raw, y_train = train[features_extended], train[y_label]
X_valid_raw, y_valid = valid[features_extended], valid[y_label]
X_test_raw,  y_test  = test[features_extended],  test[y_label]

print("Train:", X_train_raw.shape, "Valid:", X_valid_raw.shape, "Test:", X_test_raw.shape)
print("y_train NaN:", y_train.isna().sum(), "y_valid NaN:", y_valid.isna().sum(), "y_test NaN:", y_test.isna().sum())




Train: (287832, 47) Valid: (96362, 47) Test: (42071, 47)
y_train NaN: 0 y_valid NaN: 0 y_test NaN: 0


Optuna: Parameter Tuning

In [ ]:
# =========================
# Fast "Lasso-like" Linear Model (Scheme A)
# SGDRegressor + L1 (ElasticNet with l1_ratio=1.0)
# for large sparse OHE features
# =========================

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler

from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

import optuna



def eval_regression(y_true, y_pred, name="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"\n=== {name} ===")
    print(f"MAE : {mae:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"R²  : {r2:.3f}")
    return mae, rmse, r2


# -------------------------
# Preprocess
# -------------------------

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", MaxAbsScaler())
])


cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(
        handle_unknown="ignore",
        sparse_output=True,
        min_frequency=50
    ))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pipe, numeric_cols_extended),
        ("cat", cat_pipe, cat_cols_extended),
    ],
    remainder="drop",
    sparse_threshold=1.0   
)


# -------------------------
# Optuna：
# -------------------------
def objective_sgd_l1(trial):
    alpha = trial.suggest_float("alpha", 1e-6, 1e-2, log=True)   
    eps   = trial.suggest_float("epsilon", 0.1, 5.0, log=True)  
    eta0  = trial.suggest_float("eta0", 1e-4, 1e-1, log=True)    
    power_t = trial.suggest_float("power_t", 0.05, 0.5)          

    model = Pipeline([
        ("preprocess", preprocess),
        ("sgd", SGDRegressor(
            loss="epsilon_insensitive",   
            penalty="elasticnet",
            l1_ratio=1.0,                 
            alpha=alpha,
            epsilon=eps,
            learning_rate="invscaling",
            eta0=eta0,
            power_t=power_t,
            max_iter=2500,
            tol=1e-4,
            early_stopping=True,          
            validation_fraction=0.1,
            n_iter_no_change=10,
            random_state=42
        ))
    ])

    model.fit(X_train_raw, y_train)
    pred_valid = model.predict(X_valid_raw)
    mae = mean_absolute_error(y_valid, pred_valid)
    return mae


study = optuna.create_study(direction="minimize")
study.optimize(objective_sgd_l1, n_trials=25, show_progress_bar=True)

print("\n===== Fast L1 Linear (SGD) Optuna Result =====")
print("Best Valid MAE:", study.best_value)
print("Best params:", study.best_params)



[I 2025-12-13 16:14:47,051] A new study created in memory with name: no-name-355bcfa9-b804-4087-8ff0-e45635735898


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2025-12-13 16:15:28,316] Trial 0 finished with value: 10.111579388829705 and parameters: {'alpha': 8.704184843137314e-05, 'epsilon': 2.854110033224758, 'eta0': 0.010643224413051718, 'power_t': 0.35527288578558297}. Best is trial 0 with value: 10.111579388829705.
[I 2025-12-13 16:16:19,232] Trial 1 finished with value: 10.431473913035155 and parameters: {'alpha': 0.0025671860413000345, 'epsilon': 3.930577807394815, 'eta0': 0.0005918434059544146, 'power_t': 0.19267936438478578}. Best is trial 0 with value: 10.111579388829705.
[I 2025-12-13 16:16:35,066] Trial 2 finished with value: 9.744177729275247 and parameters: {'alpha': 3.82159279960969e-06, 'epsilon': 1.0562270911867166, 'eta0': 0.005536669928565463, 'power_t': 0.1234196596474363}. Best is trial 2 with value: 9.744177729275247.
[I 2025-12-13 16:17:09,853] Trial 3 finished with value: 10.394820037483155 and parameters: {'alpha': 0.00026934110522486147, 'epsilon': 2.6683353462417196, 'eta0': 0.00779126304939291, 'power_t': 0.37871

Final Model:

In [ ]:

# -------------------------
#  best params （Train+Valid），Test 
# -------------------------
best = study.best_params

linear_final = Pipeline([
    ("preprocess", preprocess),
    ("sgd", SGDRegressor(
        loss="epsilon_insensitive",
        penalty="elasticnet",
        l1_ratio=1.0,
        alpha=best["alpha"],
        epsilon=best["epsilon"],
        learning_rate="invscaling",
        eta0=best["eta0"],
        power_t=best["power_t"],
        max_iter=8000,      
        tol=1e-4,
        early_stopping=False,  # 
        random_state=42
    ))
])

X_full = pd.concat([X_train_raw, X_valid_raw], axis=0)
y_full = pd.concat([y_train, y_valid], axis=0)

linear_final.fit(X_full, y_full)

pred_valid = linear_final.predict(X_valid_raw)
pred_test  = linear_final.predict(X_test_raw)

eval_regression(y_valid, pred_valid, "Fast L1 Linear (SGD) - Valid")
eval_regression(y_test,  pred_test,  "Fast L1 Linear (SGD) - Test")



feature_names = linear_final.named_steps["preprocess"].get_feature_names_out()
coef = linear_final.named_steps["sgd"].coef_

coef_s = pd.Series(coef, index=feature_names)


zero_ratio = float((coef_s == 0).mean())
print("\nZero-coef ratio:", zero_ratio)


topN = 40
top_abs = coef_s.reindex(coef_s.abs().sort_values(ascending=False).index).head(topN)

print(f"\nTop {topN} features by |coef|:")
print(top_abs)

top_pos = coef_s.sort_values(ascending=False).head(20)
top_neg = coef_s.sort_values(ascending=True).head(20)

print("\nTop + coefficients (increase DepDelay):")
print(top_pos)

print("\nTop - coefficients (decrease DepDelay):")
print(top_neg)



=== Fast L1 Linear (SGD) - Valid ===
MAE : 9.562
RMSE: 27.147
R²  : 0.386

=== Fast L1 Linear (SGD) - Test ===
MAE : 9.899
RMSE: 29.824
R²  : 0.343

Zero-coef ratio: 0.5240128068303095

Top 40 features by |coef|:
num__prop_prev_delay_over_turn         246.870761
cat__prev_bin_>180                      82.609622
num__scheduled_turn_time                80.969673
num__short_turn_prev_delay              47.423120
num__slack_time                         42.297398
cat__prev_bin_60–180                    24.670288
num__prev_arr_delay                    -18.768845
num__carrier_delay_mean_7d              17.859547
cat__prev_arr_tbin_Hour_22              16.964447
cat__prev_bin_Early/on-time (≤0)       -15.549781
cat__crsdep_tbin_Hour_23               -14.792460
cat__prev_arr_tbin_Hour_21              14.364568
cat__prev_bin_0–15                     -14.047199
cat__prev_big_delay_1                   12.839083
cat__prev_arr_tbin_Hour_20              12.424239
num__origin_delay_mean_7d           

In [ ]:
#shrinked to 0
coef_s[coef_s==0]

num__realized slack      0.0
num__carrier_slope       0.0
num__temperature_2m      0.0
num__precipitation       0.0
num__cloud_cover         0.0
                        ... 
cat__DistanceGroup_11    0.0
cat__DayOfWeek_4         0.0
cat__short_turn_0        0.0
cat__has_rain_0          0.0
cat__has_rain_pa_0       0.0
Length: 491, dtype: float64